#### Data Cleaning and Validation

In [ ]:
import pandas as pd
import warnings
import logging

# Suppress all warnings
warnings.filterwarnings('ignore')

# Suppress Prophet/cmdstanpy logging
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)
logging.getLogger('prophet').setLevel(logging.ERROR)

In [ ]:
# Load data into dataframe
airtraffic_df = pd.read_csv("data.csv")

In [ ]:
# Look at 10 rows
airtraffic_df.head(10)

In [ ]:
# Display shape and type of df
airtraffic_df.info()

In [ ]:
# Get summary statistics for the data
airtraffic_df.describe()

HERE ARE THE VARIABLES THAT NEED TO CHANGE
- Month -> String
- AustralianPort -> String
- ForeignPort -> String
- Country -> String

In [ ]:
# Convert all of the nessesary varaibles
changed_varaibles = ["Month", "AustralianPort", "ForeignPort", "Country"]

for variable in changed_varaibles:
    airtraffic_df[variable] = airtraffic_df[variable].astype("string")

# Check if changes worked
airtraffic_df.info()

In [ ]:
# Check the frequency of null values in each column
airtraffic_df.isnull().sum()

In [ ]:
# Check the frequency of null values in each row
airtraffic_df.isnull().sum(axis=1).sort_values(ascending = False)

In [ ]:
# Check if there are duplicate rows
duplicates = airtraffic_df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

# Display duplicate rows
if duplicates.sum() > 0:
    duplicate_rows = airtraffic_df[duplicates]
    display(duplicate_rows)

In [ ]:
# Check numberical columns are positive
numerical_cols = ['Passengers_In', 'Passengers_Out', 'Passengers_Total', 
                  'Freight_In_(tonnes)', 'Freight_Out_(tonnes)', 'Freight_Total_(tonnes)',
                  'Mail_In_(tonnes)', 'Mail_Out_(tonnes)', 'Mail_Total_(tonnes)']

for col in numerical_cols:
    invalid_mask = airtraffic_df[col] < 0
    invalid_count = invalid_mask.sum()

print(f"Number of invalid rows: {invalid_count}")

In [ ]:
# Check month_num is within 1-12
invalid_months = len(airtraffic_df[(airtraffic_df['Month_num'] < 1) | (airtraffic_df['Month_num'] > 12)])

print(f"Number of rows with an invalid month_num: {invalid_months}")

### EDA

In [ ]:
# Import the custom analyzer
from traffic_analyzer import TrafficAnalyzer

# Initialize the analyzer with your cleaned data
analyzer = TrafficAnalyzer(airtraffic_df)

In [ ]:
print("\nPASSENGER TRAFFIC ANALYSIS - ALL DIRECTIONS")
print("-"*100)

# Create comparison table for passengers
passenger_total = analyzer.analyze_ranking('passengers', 'total', 'route', 5, 5)
passenger_in = analyzer.analyze_ranking('passengers', 'in', 'route', 5, 5)
passenger_out = analyzer.analyze_ranking('passengers', 'out', 'route', 5, 5)

print("\nTop 5 Passenger Routes - Comparison:")
passenger_comparison = passenger_total['data']['top'][['AustralianPort', 'ForeignPort']].copy()
passenger_comparison['Total'] = passenger_total['data']['top']['Passengers_Total']
display(passenger_comparison)

# Add in and out data
for idx, row in passenger_comparison.iterrows():
    route_data = airtraffic_df[
        (airtraffic_df['AustralianPort'] == row['AustralianPort']) & 
        (airtraffic_df['ForeignPort'] == row['ForeignPort'])
    ]
    passenger_comparison.loc[idx, 'Inbound'] = route_data['Passengers_In'].sum()
    passenger_comparison.loc[idx, 'Outbound'] = route_data['Passengers_Out'].sum()

passenger_comparison['Route'] = passenger_comparison['AustralianPort'] + ' ↔ ' + passenger_comparison['ForeignPort']
display(passenger_comparison[['Route', 'Inbound', 'Outbound', 'Total']])

print("\nBottom 5 Passenger Routes - Comparison:")
passenger_bottom_comparison = passenger_total['data']['bottom'][['AustralianPort', 'ForeignPort']].copy()
passenger_bottom_comparison['Total'] = passenger_total['data']['bottom']['Passengers_Total']

# Add in and out data for bottom routes
for idx, row in passenger_bottom_comparison.iterrows():
    route_data = airtraffic_df[
        (airtraffic_df['AustralianPort'] == row['AustralianPort']) & 
        (airtraffic_df['ForeignPort'] == row['ForeignPort'])
    ]
    passenger_bottom_comparison.loc[idx, 'Inbound'] = route_data['Passengers_In'].sum()
    passenger_bottom_comparison.loc[idx, 'Outbound'] = route_data['Passengers_Out'].sum()

passenger_bottom_comparison['Route'] = passenger_bottom_comparison['AustralianPort'] + ' ↔ ' + passenger_bottom_comparison['ForeignPort']
display(passenger_bottom_comparison[['Route', 'Inbound', 'Outbound', 'Total']])

In [ ]:
print("\nFREIGHT TRAFFIC ANALYSIS - ALL DIRECTIONS")
print("-"*100)

# Create comparison table for freight
freight_total = analyzer.analyze_ranking('freight', 'total', 'route', 5, 5)
freight_in = analyzer.analyze_ranking('freight', 'in', 'route', 5, 5)
freight_out = analyzer.analyze_ranking('freight', 'out', 'route', 5, 5)

print("\nTop 5 Freight Routes - Comparison:")
freight_comparison = freight_total['data']['top'][['AustralianPort', 'ForeignPort']].copy()
freight_comparison['Total'] = freight_total['data']['top']['Freight_Total_(tonnes)']

# Add in and out data
for idx, row in freight_comparison.iterrows():
    route_data = airtraffic_df[
        (airtraffic_df['AustralianPort'] == row['AustralianPort']) & 
        (airtraffic_df['ForeignPort'] == row['ForeignPort'])
    ]
    freight_comparison.loc[idx, 'Inbound'] = route_data['Freight_In_(tonnes)'].sum()
    freight_comparison.loc[idx, 'Outbound'] = route_data['Freight_Out_(tonnes)'].sum()

freight_comparison['Route'] = freight_comparison['AustralianPort'] + ' ↔ ' + freight_comparison['ForeignPort']
display(freight_comparison[['Route', 'Inbound', 'Outbound', 'Total']])

print("\nBottom 5 Freight Routes - Comparison:")
freight_bottom_comparison = freight_total['data']['bottom'][['AustralianPort', 'ForeignPort']].copy()
freight_bottom_comparison['Total'] = freight_total['data']['bottom']['Freight_Total_(tonnes)']

# Add in and out data for bottom routes
for idx, row in freight_bottom_comparison.iterrows():
    route_data = airtraffic_df[
        (airtraffic_df['AustralianPort'] == row['AustralianPort']) & 
        (airtraffic_df['ForeignPort'] == row['ForeignPort'])
    ]
    freight_bottom_comparison.loc[idx, 'Inbound'] = route_data['Freight_In_(tonnes)'].sum()
    freight_bottom_comparison.loc[idx, 'Outbound'] = route_data['Freight_Out_(tonnes)'].sum()

freight_bottom_comparison['Route'] = freight_bottom_comparison['AustralianPort'] + ' ↔ ' + freight_bottom_comparison['ForeignPort']
display(freight_bottom_comparison[['Route', 'Inbound', 'Outbound', 'Total']])

In [ ]:
print("\nMAIL TRAFFIC ANALYSIS - ALL DIRECTIONS")
print("-"*100)

# Create comparison table for mail
mail_total = analyzer.analyze_ranking('mail', 'total', 'route', 5, 5)
mail_in = analyzer.analyze_ranking('mail', 'in', 'route', 5, 5)
mail_out = analyzer.analyze_ranking('mail', 'out', 'route', 5, 5)

print("\nTop 5 Mail Routes - Comparison:")
mail_comparison = mail_total['data']['top'][['AustralianPort', 'ForeignPort']].copy()
mail_comparison['Total'] = mail_total['data']['top']['Mail_Total_(tonnes)']

# Add in and out data
for idx, row in mail_comparison.iterrows():
    route_data = airtraffic_df[
        (airtraffic_df['AustralianPort'] == row['AustralianPort']) & 
        (airtraffic_df['ForeignPort'] == row['ForeignPort'])
    ]
    mail_comparison.loc[idx, 'Inbound'] = route_data['Mail_In_(tonnes)'].sum()
    mail_comparison.loc[idx, 'Outbound'] = route_data['Mail_Out_(tonnes)'].sum()

mail_comparison['Route'] = mail_comparison['AustralianPort'] + ' ↔ ' + mail_comparison['ForeignPort']
display(mail_comparison[['Route', 'Inbound', 'Outbound', 'Total']])

print("\nBottom 5 Mail Routes - Comparison:")
mail_bottom_comparison = mail_total['data']['bottom'][['AustralianPort', 'ForeignPort']].copy()
mail_bottom_comparison['Total'] = mail_total['data']['bottom']['Mail_Total_(tonnes)']

# Add in and out data for bottom routes
for idx, row in mail_bottom_comparison.iterrows():
    route_data = airtraffic_df[
        (airtraffic_df['AustralianPort'] == row['AustralianPort']) & 
        (airtraffic_df['ForeignPort'] == row['ForeignPort'])
    ]
    mail_bottom_comparison.loc[idx, 'Inbound'] = route_data['Mail_In_(tonnes)'].sum()
    mail_bottom_comparison.loc[idx, 'Outbound'] = route_data['Mail_Out_(tonnes)'].sum()

mail_bottom_comparison['Route'] = mail_bottom_comparison['AustralianPort'] + ' ↔ ' + mail_bottom_comparison['ForeignPort']
display(mail_bottom_comparison[['Route', 'Inbound', 'Outbound', 'Total']])

In [ ]:
print("\nPASSENGER TEMPORAL TRENDS ANALYSIS")
print("-"*100)

# Overall traffic trends over time
temporal_result = analyzer.analyze_temporal('passengers', 'total', groupby_level='total')

print("\nMonthly Passenger Traffic Trends:")
display(temporal_result['data'][['Month_dt', 'Passengers_Total', 'MoM_Growth', '3M_MA']].tail(12))

print("\nKey Metrics:")
print(f"  Monthly Growth Rate: {temporal_result['statistics']['trend_pct_monthly']:.2f}%")
print(f"  Total Period Growth: {((temporal_result['statistics']['end_value'] - temporal_result['statistics']['start_value']) / temporal_result['statistics']['start_value'] * 100):.1f}%")
print(f"  Volatility (CV): {temporal_result['statistics']['cv']:.2f}")

In [ ]:
print("\nFREIGHT TEMPORAL TRENDS ANALYSIS")
print("-"*100)

# Overall traffic trends over time
temporal_result = analyzer.analyze_temporal('freight', 'total', groupby_level='total')

print("\nMonthly Freight Traffic Trends:")
display(temporal_result['data'][['Month_dt', 'Freight_Total_(tonnes)', 'MoM_Growth', '3M_MA']].tail(12))

print("\nKey Metrics:")
print(f"  Monthly Growth Rate: {temporal_result['statistics']['trend_pct_monthly']:.2f}%")
print(f"  Total Period Growth: {((temporal_result['statistics']['end_value'] - temporal_result['statistics']['start_value']) / temporal_result['statistics']['start_value'] * 100):.1f}%")
print(f"  Volatility (CV): {temporal_result['statistics']['cv']:.2f}")

In [ ]:
print("\nMAIL TEMPORAL TRENDS ANALYSIS")
print("-"*100)

# Overall traffic trends over time
temporal_result = analyzer.analyze_temporal('mail', 'total', groupby_level='total')

print("\nMonthly Freight Traffic Trends:")
display(temporal_result['data'][['Month_dt', 'Mail_Total_(tonnes)', 'MoM_Growth', '3M_MA']].tail(12))

print("\nKey Metrics:")
print(f"  Monthly Growth Rate: {temporal_result['statistics']['trend_pct_monthly']:.2f}%")
print(f"  Total Period Growth: {((temporal_result['statistics']['end_value'] - temporal_result['statistics']['start_value']) / temporal_result['statistics']['start_value'] * 100):.1f}%")
print(f"  Volatility (CV): {temporal_result['statistics']['cv']:.2f}")

In [ ]:
print("\nPASSENGER SEASONAL PATTERNS ANALYSIS")
print("-"*100)

seasonal_result = analyzer.analyze_seasonal('passengers', 'total', 'total')

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
seasonal_df = seasonal_result['data'].copy()
seasonal_df['Month'] = [month_names[i-1] for i in seasonal_df['Month_Num']]

print("\nAverage Traffic by Month:")
display(seasonal_df[['Month', 'Passengers_Total']])

print("Seasonality Metrics:")
print(f"  Peak Month: {month_names[seasonal_result['statistics']['peak_month']-1]}")
print(f"  Trough Month: {month_names[seasonal_result['statistics']['trough_month']-1]}")
print(f"  Seasonal Strength: {seasonal_result['statistics']['seasonal_strength']:.1%}")

In [ ]:
print("\nFREIGHT SEASONAL PATTERNS ANALYSIS")
print("-"*100)

seasonal_result = analyzer.analyze_seasonal('freight', 'total', 'total')

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
seasonal_df = seasonal_result['data'].copy()
seasonal_df['Month'] = [month_names[i-1] for i in seasonal_df['Month_Num']]

print("\nAverage Traffic by Month:")
display(seasonal_df[['Month', 'Freight_Total_(tonnes)']])

print("Seasonality Metrics:")
print(f"  Peak Month: {month_names[seasonal_result['statistics']['peak_month']-1]}")
print(f"  Trough Month: {month_names[seasonal_result['statistics']['trough_month']-1]}")
print(f"  Seasonal Strength: {seasonal_result['statistics']['seasonal_strength']:.1%}")

In [ ]:
print("\nMAIL SEASONAL PATTERNS ANALYSIS")
print("-"*100)

seasonal_result = analyzer.analyze_seasonal('mail', 'total', 'total')

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
seasonal_df = seasonal_result['data'].copy()
seasonal_df['Month'] = [month_names[i-1] for i in seasonal_df['Month_Num']]

print("\nAverage Traffic by Month:")
display(seasonal_df[['Month', 'Mail_Total_(tonnes)']])

print("Seasonality Metrics:")
print(f"  Peak Month: {month_names[seasonal_result['statistics']['peak_month']-1]}")
print(f"  Trough Month: {month_names[seasonal_result['statistics']['trough_month']-1]}")
print(f"  Seasonal Strength: {seasonal_result['statistics']['seasonal_strength']:.1%}")

In [ ]:
print("\nPASSENGER GEOGRAPHICAL PATTERNS ANALYSIS")
print("-"*100)

# By Country
country_result = analyzer.analyze_ranking('passengers', 'total', 'country', 5, 5)

print("\nTop 10 Countries by Passenger Traffic:")
country_df = country_result['data']['top'].copy()
country_df['% of Total'] = (country_df['Passengers_Total'] / country_df['Passengers_Total'].sum() * 100)
display(country_df[['Label', 'Passengers_Total', '% of Total']])

# By Australian Port
port_result = analyzer.analyze_ranking('passengers', 'total', 'port', 5, 5)

print("\nAustralian Port Hub Analysis:")
display(port_result['data']['top'][['Label', 'Passengers_Total']])

In [ ]:
print("\nFREIGHT GEOGRAPHICAL PATTERNS ANALYSIS")
print("-"*100)

# By Country
country_result = analyzer.analyze_ranking('freight', 'total', 'country', 5, 5)

print("\nTop 10 Countries by Freight Traffic:")
country_df = country_result['data']['top'].copy()
country_df['% of Total'] = (country_df['Freight_Total_(tonnes)'] / country_df['Freight_Total_(tonnes)'].sum() * 100)
display(country_df[['Label', 'Freight_Total_(tonnes)', '% of Total']])

# By Australian Port (Hub Analysis)
port_result = analyzer.analyze_ranking('freight', 'total', 'port', 5, 5)

print("\nAustralian Port Hub Analysis:")
display(port_result['data']['top'][['Label', 'Freight_Total_(tonnes)']])

In [ ]:
print("\nMAIL GEOGRAPHICAL PATTERNS ANALYSIS")
print("-"*100)

# By Country
country_result = analyzer.analyze_ranking('mail', 'total', 'country', 5, 5)

print("\nTop 10 Countries by Mail Traffic:")
country_df = country_result['data']['top'].copy()
country_df['% of Total'] = (country_df['Mail_Total_(tonnes)'] / country_df['Mail_Total_(tonnes)'].sum() * 100)
display(country_df[['Label', 'Mail_Total_(tonnes)', '% of Total']])

# By Australian Port (Hub Analysis)
port_result = analyzer.analyze_ranking('mail', 'total', 'port', 5, 5)

print("\nAustralian Port Hub Analysis:")
display(port_result['data']['top'][['Label', 'Mail_Total_(tonnes)']])

In [ ]:
print("\nPASSENGER TRAFFIC BALANCE ANALYSIS")
print("-"*100)

balance_result = analyzer.analyze_balance('passengers', groupby_level='route')

print("\nMost Imbalanced Routes:")
balance_df = balance_result['data'].head(10).copy()
balance_df['Status'] = balance_df['Balance_Ratio'].apply(
    lambda x: 'Balanced' if 0.8 <= x <= 1.2 else 'Imbalanced' if pd.notna(x) else 'N/A'
)
display(balance_df[['Label', 'Passengers_In', 'Passengers_Out', 'Balance_Ratio', 'Status']])

In [ ]:
print("\nFREIGHT TRAFFIC BALANCE ANALYSIS")
print("-"*100)

balance_result = analyzer.analyze_balance('freight', groupby_level='route')

print("\nMost Imbalanced Routes:")
balance_df = balance_result['data'].head(10).copy()
balance_df['Status'] = balance_df['Balance_Ratio'].apply(
    lambda x: 'Balanced' if 0.8 <= x <= 1.2 else 'Imbalanced' if pd.notna(x) else 'N/A'
)
display(balance_df[['Label', 'Freight_In_(tonnes)', 'Freight_Out_(tonnes)', 'Balance_Ratio', 'Status']])

In [ ]:
print("\nMAIL TRAFFIC BALANCE ANALYSIS")
print("-"*100)

balance_result = analyzer.analyze_balance('mail', groupby_level='route')

print("\nMost Imbalanced Routes:")
balance_df = balance_result['data'].head(10).copy()
balance_df['Status'] = balance_df['Balance_Ratio'].apply(
    lambda x: 'Balanced' if 0.8 <= x <= 1.2 else 'Imbalanced' if pd.notna(x) else 'N/A'
)
display(balance_df[['Label', 'Mail_In_(tonnes)', 'Mail_Out_(tonnes)', 'Balance_Ratio', 'Status']])

In [ ]:
print("\nGROWTH OPPORTUNITIES AND RISK ANALYSIS")
print("-"*100)

# High growth routes
opportunities = analyzer.find_opportunities(min_growth_rate=3.0)
if len(opportunities) > 0:
    print("\nHigh Growth Routes (>3% monthly growth):")
    display(opportunities[['route', 'monthly_growth', 'period_growth', 'current_volume', 'potential']].head(10))

# Declining routes
risks = analyzer.identify_risks(max_decline_rate=-2.0)
if len(risks) > 0:
    print("\nDeclining Routes (<2% monthly decline):")
    display(risks[['route', 'monthly_decline', 'period_decline', 'current_volume', 'risk_level']].head(10))

print("ROUTES SELECTED FOR FORECASTING:\n")

# Top performers
print("1. HIGH-VOLUME ROUTES (Critical for business):")
top_routes = passenger_total['data']['top'].head(3)
for idx, row in top_routes.iterrows():
    print(f"   • {row['AustralianPort']} ↔ {row['ForeignPort']} - {row['Passengers_Total']:,.0f} total passengers")

# High growth opportunities 
if len(opportunities) > 0:
    print("\n2. HIGH-GROWTH OPPORTUNITIES (Potential investment targets):")
    for idx, row in opportunities.head(2).iterrows():
        print(f"   • {row['route']} - {row['monthly_growth']:.1f}% monthly growth")

# Declining routes
if len(risks) > 0:
    print("\n3. DECLINING ROUTES (Risk management needed):")
    for idx, row in risks.head(2).iterrows():
        print(f"   • {row['route']} - {row['monthly_decline']:.1f}% monthly decline")

### MODEL SELECTION

In [ ]:
from route_forcaster import RouteForecaster, FutureForecaster

routes = ['Sydney-Auckland', 'Sydney-Singapore', 'Sydney-Tokyo']
summary_data = []

for route in routes:
    # Run analysis
    rf = RouteForecaster(airtraffic_df, route, 'passengers', 'total')
    rf.run_complete_analysis()
    
    # Generate future forecasts
    ff = FutureForecaster(rf)
    ff.get_best_model_forecast(n_months=12)
    
    # Plot visualization
    ff.plot_future_forecast(model_name=rf.metrics and min(rf.metrics.keys(), key=lambda x: rf.metrics[x]['MAPE']))
    
    # Summary report
    ff.summary_report()
    
    # Collect for table
    best_model = min(rf.metrics.keys(), key=lambda x: rf.metrics[x]['MAPE'])
    forecast = ff.future_predictions[best_model]['forecast']
    
    summary_data.append({
        'Route': route,
        'Best Model': best_model,
        'MAPE (%)': rf.metrics[best_model]['MAPE'],
        '6-Month Avg': forecast[:6].mean(),
        '12-Month Avg': forecast.mean(),
        'Growth (%)': ((forecast[-1] - forecast[0]) / forecast[0] * 100)
    })

# Display summary table
summary_df = pd.DataFrame(summary_data)
print("\nSUMMARY TABLE")
display(summary_df)